In [2]:
import json
import urllib.request as urlreq
import time, datetime
import pandas as pd
import codecs
# import sys
# print(sys.version)

In [3]:
def get_stock_list(fname):
    """Open .csv file containing list of stocks, and their weights in the ETF"""
    stock_list = pd.read_csv(fname)
    return stock_list

In [4]:
def get_quotes(stock_list):
    """Get quote for each stock in list, return list of responses from the API"""
    url = "http://finance.google.com/finance/info?client=ig&q="
    for stock in stock_list:
        url = url + '{},'.format(stock)
    raw_data = urlreq.urlopen(url).read().decode('utf8')

    return json.loads(raw_data[3:])

In [5]:
def calc_inav(stock_list, dow_divisor):
    """Calculate Indicative Net Asset Value of DJT. Returns calculated value, and value for
    the DJT received from API call"""
    quotes = get_quotes(stock_list['Ticker'])

    inav_df = stock_list.set_index('Ticker')

    for i, stock in enumerate(quotes):
        inav_df.set_value(stock['t'], 'Last_Price', float(stock['l'].replace(',','')))

    inav = inav_df[(inav_df.index != 'DJT')]['Last_Price'].sum(axis=0) / dow_divisor

    market_index = inav_df[(inav_df.index == 'DJT')]['Last_Price'].iloc[0]

    output = {'time': datetime.datetime.now(), 'inav': inav, 'market_index': market_index}

    return output

In [6]:
def plot():
    pass

In [7]:
def run_app(stock_list):
    """Will run the program continuously until user exits, getting new data every 1 secs"""
    try:
        values_df = pd.DataFrame()
        while True:
            values = calc_inav(stock_list, dow_divisor)
            temp_df = pd.DataFrame([values]).set_index('time')
            values_df = pd.concat([values_df, temp_df])
            
            time.sleep(1)            

    except KeyboardInterrupt:
        print('Exiting...')
        print(values_df)
        exit()

In [8]:
if __name__ == '__main__':

    '''DOW divisor source: http://online.barrons.com/mdc/public/page/9_3022-djiahourly.html?mod=mdc_h_usshl'''
    dow_divisor = 0.16416809180007

    stock_list = get_stock_list('DJT_Component_Weights.csv')

    """Runs the program continuously until exit"""
    run_app(stock_list)

Exiting...
                                   inav  market_index
time                                                 
2017-02-14 23:03:05.888575  9444.222583       9444.22
2017-02-14 23:03:07.005301  9444.222583       9444.22
2017-02-14 23:03:09.043831  9444.222583       9444.22
2017-02-14 23:03:10.155877  9444.222583       9444.22
2017-02-14 23:03:11.268175  9444.222583       9444.22
